In [71]:
import pandas as pd
from datetime import datetime
from IPython.display import display

# ID of the Google Sheets file
file_id = "1KLmqRbECQwOUvOpU3v60PKWQsfVrpmau5yDKcVkOXMw"
url = f"https://docs.google.com/spreadsheets/d/{file_id}/export?format=csv&gid=0"

# Read the sheet with a row limit to avoid reading extra blank rows
df = pd.read_csv(url, header=2, nrows=86)

# Clean up column names to remove line breaks and extra spaces
df.columns = df.columns.str.replace('\n', ' ', regex=True).str.replace('  ', ' ').str.strip()

# Convert the date columns with dayfirst=True to match the Brazilian date format
df['INÍCIO DA VIGÊNCIA'] = pd.to_datetime(df['INÍCIO DA VIGÊNCIA'], errors='coerce', dayfirst=True)
df['FIM DA VIGÊNCIA'] = pd.to_datetime(df['FIM DA VIGÊNCIA'], errors='coerce', dayfirst=True)
df['DATA FINAL PARA ENCAMINHAMENTO'] = pd.to_datetime(df['DATA FINAL PARA ENCAMINHAMENTO'], errors='coerce', dayfirst=True)

# Current date for comparison
current_date = datetime.now()
current_year = current_date.year

# Total de TEDs firmados
teds_firmados_total = df[df['INÍCIO DA VIGÊNCIA'].notna()].shape[0]

# Contagem de TEDs finalizados (FIM DA VIGÊNCIA < data atual)
teds_finalizados_total = df[df['FIM DA VIGÊNCIA'] < current_date].shape[0]

# Contagem de TEDs vigentes (FIM DA VIGÊNCIA >= data atual)
teds_vigentes_total = df[df['FIM DA VIGÊNCIA'] >= current_date].shape[0]

# Contagem adicional para conferência
# teds_vigentes_calculado = teds_firmados_total - teds_finalizados_total

# Verificação das contagens para consistência
print("Resumo das Contagens Ajustado:")
print(f"Total de TEDs Firmados: {teds_firmados_total}")
print(f"Total de TEDs Finalizados: {teds_finalizados_total}")
# print(f"Total de TEDs Vigentes (calculado): {teds_vigentes_calculado}")
print(f"Total de TEDs Vigentes: {teds_vigentes_total}")
print()

# Contagem de TEDs firmados por ano
firmados_por_ano = df['INÍCIO DA VIGÊNCIA'].dt.year.value_counts().sort_index()
firmados_por_ano.index = firmados_por_ano.index.astype(int)

# Contagem de TEDs finalizados por ano
finalizados_por_ano = df[df['FIM DA VIGÊNCIA'] < current_date]['FIM DA VIGÊNCIA'].dt.year.value_counts().sort_index()
finalizados_por_ano.index = finalizados_por_ano.index.astype(int)

# Criar tabela com TEDs firmados e finalizados por ano
tabela_ano = pd.DataFrame({
    "Ano": firmados_por_ano.index,
    "TEDs Firmados": firmados_por_ano.values,
    "TEDs Finalizados": finalizados_por_ano.reindex(firmados_por_ano.index, fill_value=0).values
})

# Aplicar estilo e exibir tabela de contagem de TEDs por ano
tabela_ano_estilizada = tabela_ano.style.set_table_styles(
    [{'selector': 'table', 'props': [('border', '2px solid black'), ('border-collapse', 'collapse')]},
     {'selector': 'th, td', 'props': [('border', '1px solid black'), ('padding', '5px')]}]
).set_caption("Contagem de TEDs por Ano")

# Contagem de TEDs no período de prestação de contas
teds_prestacao_contas = df[(df['FIM DA VIGÊNCIA'] < current_date) & 
                           (df['DATA FINAL PARA ENCAMINHAMENTO'] > current_date)].shape[0]

# Criar tabela de status atual com TEDs vigentes e no período de prestação de contas
tabela_status = pd.DataFrame({
    "Status": ["TEDs Vigentes", "TEDs no Período de Prestação"],
    "Quantidade": [teds_vigentes_total, teds_prestacao_contas]
})

# Aplicar estilo e exibir tabela de status
tabela_status_estilizada = tabela_status.style.set_table_styles(
    [{'selector': 'table', 'props': [('border', '2px solid black'), ('border-collapse', 'collapse')]},
     {'selector': 'th, td', 'props': [('border', '1px solid black'), ('padding', '5px')]}]
).set_caption("Status Atual de TEDs")

# Filtrar e exibir lista de TEDs no período de prestação de contas
teds_prestacao_contas_lista = df[(df['FIM DA VIGÊNCIA'] < current_date) & 
                                 (df['DATA FINAL PARA ENCAMINHAMENTO'] > current_date)][['Seq.', 'TED/ANO']]

# Redefinir o índice para remover a coluna de índice original
teds_prestacao_contas_lista.reset_index(drop=True, inplace=True)

# Aplicar estilo e exibir tabela de TEDs no período de prestação de contas
teds_prestacao_contas_lista_estilizada = teds_prestacao_contas_lista.style.set_table_styles(
    [{'selector': 'table', 'props': [('border', '2px solid black'), ('border-collapse', 'collapse')]},
     {'selector': 'th, td', 'props': [('border', '1px solid black'), ('padding', '5px')]}]
).set_caption("Lista de TEDs no Período de Prestação de Contas")

# Exibir as tabelas
display(tabela_ano_estilizada)
display(tabela_status_estilizada)
display(teds_prestacao_contas_lista_estilizada)


Resumo das Contagens Ajustado:
Total de TEDs Firmados: 84
Total de TEDs Finalizados: 43
Total de TEDs Vigentes: 41



,Ano,TEDs Firmados,TEDs Finalizados
0,2018,2,0
1,2019,9,0
2,2020,6,0
3,2021,8,0
4,2022,21,0
5,2023,29,24
6,2024,9,19


,Status,Quantidade
0,TEDs Vigentes,41
1,TEDs no Período de Prestação,2


,Seq.,TED/ANO
0,9,9771/2020
1,23,11377


In [3]:
import streamlit as st
import pandas as pd
from datetime import datetime

# ID of the Google Sheets file
file_id = "1KLmqRbECQwOUvOpU3v60PKWQsfVrpmau5yDKcVkOXMw"
url = f"https://docs.google.com/spreadsheets/d/{file_id}/export?format=csv&gid=0"

# Título do aplicativo
st.title("Análise de TEDs")

# Carregar e processar os dados
df = pd.read_csv(url, header=2, nrows=86)
df.columns = df.columns.str.replace('\n', ' ', regex=True).str.replace('  ', ' ').str.strip()

# Conversão de datas
df['INÍCIO DA VIGÊNCIA'] = pd.to_datetime(df['INÍCIO DA VIGÊNCIA'], errors='coerce', dayfirst=True)
df['FIM DA VIGÊNCIA'] = pd.to_datetime(df['FIM DA VIGÊNCIA'], errors='coerce', dayfirst=True)
df['DATA FINAL PARA ENCAMINHAMENTO'] = pd.to_datetime(df['DATA FINAL PARA ENCAMINHAMENTO'], errors='coerce', dayfirst=True)

# Data atual
current_date = datetime.now()

# Cálculos de contagem
teds_firmados_total = df[df['INÍCIO DA VIGÊNCIA'].notna()].shape[0]
teds_finalizados_total = df[df['FIM DA VIGÊNCIA'] < current_date].shape[0]
teds_vigentes_total = df[df['FIM DA VIGÊNCIA'] >= current_date].shape[0]
teds_vigentes_calculado = teds_firmados_total - teds_finalizados_total

# Exibir resumo das contagens
st.subheader("Resumo das Contagens")
st.write(f"Total de TEDs Firmados: {teds_firmados_total}")
st.write(f"Total de TEDs Finalizados: {teds_finalizados_total}")
st.write(f"Total de TEDs Vigentes (calculado): {teds_vigentes_calculado}")
st.write(f"Total de TEDs Vigentes (diretamente contado): {teds_vigentes_total}")

# Contagem de TEDs por ano
firmados_por_ano = df['INÍCIO DA VIGÊNCIA'].dt.year.value_counts().sort_index()
finalizados_por_ano = df[df['FIM DA VIGÊNCIA'] < current_date]['FIM DA VIGÊNCIA'].dt.year.value_counts().sort_index()
tabela_ano = pd.DataFrame({
    "Ano": firmados_por_ano.index,
    "TEDs Firmados": firmados_por_ano.values,
    "TEDs Finalizados": finalizados_por_ano.reindex(firmados_por_ano.index, fill_value=0).values
})
st.subheader("Contagem de TEDs por Ano")
st.dataframe(tabela_ano)

# Status atual de TEDs
teds_prestacao_contas = df[(df['FIM DA VIGÊNCIA'] < current_date) & 
                           (df['DATA FINAL PARA ENCAMINHAMENTO'] > current_date)].shape[0]
tabela_status = pd.DataFrame({
    "Status": ["TEDs Vigentes", "TEDs no Período de Prestação"],
    "Quantidade": [teds_vigentes_total, teds_prestacao_contas]
})
st.subheader("Status Atual de TEDs")
st.dataframe(tabela_status)

# Lista de TEDs no período de prestação de contas
teds_prestacao_contas_lista = df[(df['FIM DA VIGÊNCIA'] < current_date) & 
                                 (df['DATA FINAL PARA ENCAMINHAMENTO'] > current_date)][['Seq.', 'TED/ANO']]
st.subheader("Lista de TEDs no Período de Prestação de Contas")
st.dataframe(teds_prestacao_contas_lista)


2024-11-13 16:58:38.166 
  command:

    streamlit run /Applications/anaconda3/lib/python3.8/site-packages/ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator()